In [ ]:
from GlobalVariables import global_vars as gv
import pygui as pgui
import pandas as pd
pgui.setup_interface()
pgui.db_list.value = 'tmind'
event = {'name': 'row_added', 'index': 1, 'source': 'gui'}

# Make a new fow for the existing dataframe
df = pgui.grid_manager.filterGridObject.qgrid_filter.get_changed_df()
pgui.grid_manager.filterGridObject.selection_widgets['Data Set'].value = 'Multi Index'
pgui.grid_manager.filterGridObject.selection_widgets['Creature'].value = ['Necromancer']

new_row = pd.DataFrame({
            'Modifier': [''],
            'op1': [''],
            'Creature': [''],
            'op2': [''],
            'Spell': ['River Of Souls'],
            'Data Set': ['Multi Index'],
            'Active': [False]
        })
new_df = pd.concat([df, new_row], ignore_index=True)
pgui.grid_manager.filterGridObject.qgrid_filter.df = new_df
pgui.grid_manager.filterGridObject.grid_filter_on_row_added(event, pgui.grid_manager.filterGridObject.qgrid_filter)

In [ ]:
import pygui as pgui
pgui.setup_interface()
pgui.username_widget.value = 'strudel'
#change = {'name': 'value', 'old': '', 'new': 'tmind'}
#pgui.reload_data_on_click(None,'Load Decks/Fusions')
pgui.reload_data_on_click(None,'Create all Fusions')
#pgui.reload_data_on_click(None,'Generate Dataframe')
#pgui.db_list.value = 'tmind'
#pgui.handle_db_list_change(change)

#GridManager.get_forgeborn_abilities()

#event = {'name': 'row_added', 'index': 1, 'source': 'gui'}
#pgui.grid_manager.filterGridObject.grid_filter_on_row_added(event, pgui.grid_manager.filterGridObject.qgrid_filter)
#pgui.reload_data_on_click(None,'Decks')
if False:
    #pgui.db_list.value = 'tmind'
    #event = {'name': 'selection_changed', 'old': [], 'new': [1], 'source': 'gui'}

    event = {'name': 'cell edited', 'index': 0, 'column': 'Active', 'new': False, 'source': 'filter'}
    pgui.grid_manager.filterGridObject.grid_filter_on_cell_edit(event,pgui.grid_manager.filterGridObject.qgrid_filter)
    #collection_grid = pgui.qm.grids.get('collection')
    #df = pgui.generate_deck_content_dataframe(event, collection_grid.main_widget)
    #df = pgui.generate_deck_statistics_dataframe()

In [2]:
import pygui as pgui

def select_row_in_qgrid(GridManager, row_index=0, grid_manager=None):
    """
    Programmatically selects a row in the qgrid widget and triggers the selection event.
    """
    # Ensure the deck content grid has data
    grid_df = GridManager.get_grid_df("filtered_grid_0")

    if grid_df is not None and not grid_df.empty:
        # Check if the specified row index is valid
        if 0 <= row_index < len(grid_df):
            # Programmatically select the row
            GridManager.grids["filtered_grid_0"].main_widget.change_selection([row_index])

            # Trigger the selection event manually
            event = {
                'new': [row_index]  # 'new' key contains the selected row index
            }

            # Call the handler if grid_manager is provided
            if grid_manager:
                grid_manager.update_deck_content(event, GridManager.grids["filtered_grid_0"].main_widget)
        else:
            print(f"Invalid row index {row_index}. Must be between 0 and {len(grid_df) - 1}.")
    else:
        print("No data in the grid to select.")

def trigger_sort_on_column(deck_content_grid, column_name, grid_manager=None):
    """
    Programmatically triggers a sort on a specific column in the qgrid widget.
    """
    # Get the grid DataFrame
    
    grid_df = deck_content_grid.get_changed_df()
    
    if grid_df is not None and column_name in grid_df.columns:
                
        # Trigger the sort_changed event manually
        event = {
            'new': {
                'column': column_name,
                'ascending': True  # Adjust this based on desired sort order
            }
        }
        
        # Call the grid manager's sort handler
        if grid_manager:
            grid_manager.handle_sort_changed(event, deck_content_grid)
    else:
        print(f"Column '{column_name}' not found in the grid.")


# Initialize the interface
pgui.setup_interface()
pgui.db_list.value = 'strudel'

# Assuming you want to select the first row in the filter grid and trigger an update
select_row_in_qgrid(pgui.grid_manager.qm, 0, pgui.grid_manager)

# Trigger a sort on a specific column (e.g., "Name")
trigger_sort_on_column(pgui.grid_manager.deck_content_grid, "Spell", pgui.grid_manager)

#pgui.reload_data_on_click(pgui.button_load, 'Fusions')

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Sample DataFrame for demonstration purposes
df = pd.DataFrame({
    col: [i % 5 for i in range(100)] for col in [
        'Name', 'registeredDate', 'UpdatedAt', 'xp', 'elo', 'level', 'pExpiry', 'cardSetNo', 'faction', 'forgebornId',
        'Deck A', 'Deck B', 'faction', 'crossFaction', 'forgebornId',
        'A1', 'H1', 'A2', 'H2', 'A3', 'H3', 'Creatures', 'Spells', 'Exalts',
        'Beast Synergy', 'Mage Synergy', 'Robot Synergy', 'Scientist Synergy', 'Spirit Synergy', 'Warrior Synergy',
        'Zombie Synergy', 'Activate Combo', 'Upgrade Synergy', 'Face Burn Combo', 'Removal Synergy', 'Silence Combo',
        'Annoying', 'Fun', 'Cool', 'Spicy', 'Last Winter'
    ]
})

# Groups of columns organized by categories
groups = {
    'Deck Data': ['Name', 'registeredDate', 'UpdatedAt', 'xp', 'elo', 'level', 'pExpiry', 'cardSetNo', 'faction', 'forgebornId'],
    'Fusion Data': ['Name', 'Deck A', 'Deck B', 'faction', 'crossFaction', 'forgebornId'],
    'Statistical Data': ['A1', 'H1', 'A2', 'H2', 'A3', 'H3', 'Creatures', 'Spells', 'Exalts'],
    'Synergies': [col for col in df.columns if 'Synergy' in col],
    'Combos': [col for col in df.columns if 'Combo' in col],
    'Action and Ability Attributes': ['Activate', 'Ready', 'Free', 'Upgrade', 'Face Burn', 'Removal', 'Silence', 'Deploy', 'Slay', 'Reanimate'] + 
                                      [col for col in df.columns if any(term in col for term in ['Synergy', 'Combo'])],
    'Miscellaneous Attributes': ['Annoying', 'Fun', 'Cool', 'Spicy', 'Last Winter']
}

# Widget to select groups
group_selector = widgets.SelectMultiple(
    options=groups.keys(),
    description="",
    disabled=False
)
group_selector_label = widgets.Label("Select Groups:")

# Widget to display columns based on group selection
column_selector = widgets.SelectMultiple(
    options=[],
    description="",
    disabled=False
)
column_selector_label = widgets.Label("Select Columns:")

def update_column_selector(*args):
    selected_groups = group_selector.value
    selected_columns = [col for group in selected_groups for col in groups[group]]
    column_selector.options = selected_columns

group_selector.observe(update_column_selector, 'value')

# Toggle buttons for show/hide
toggle_buttons = widgets.ToggleButtons(
    options=['Show Selected', 'Hide Selected'],
    description='',
    disabled=False
)
toggle_buttons_label = widgets.Label("Set Visibility:")

# Output widget to display changes and messages
output = widgets.Output()

def toggle_visibility(change):
    columns_to_toggle = column_selector.value
    action = change.new

    with output:
        clear_output(wait=True)
        if action == 'Show Selected':
            print(f"Showing columns: {', '.join(columns_to_toggle)}")
        elif action == 'Hide Selected':
            print(f"Hiding columns: {', '.join(columns_to_toggle)}")

toggle_buttons.observe(toggle_visibility, 'value')

# Button to lock/unlock configurations
lock_button = widgets.Button(
    description='Lock',
    disabled=False,
    button_style='',
    tooltip='Click to lock/unlock the current configuration',
    icon='lock'
)
lock_button_label = widgets.Label("Lock/Unlock Configurations:")

def lock_configuration(b):
    if lock_button.description == 'Lock':
        lock_button.description = 'Unlock'
        lock_button.icon = 'unlock'
        group_selector.disabled = True
        column_selector.disabled = True
        toggle_buttons.disabled = True
    else:
        lock_button.description = 'Lock'
        lock_button.icon = 'lock'
        group_selector.disabled = False
        column_selector.disabled = False
        toggle_buttons.disabled = False

    with output:
        clear_output()
        print(f"Configuration {'locked' if lock_button.description == 'Unlock' else 'unlocked'}.")

lock_button.on_click(lock_configuration)

# Arranging widgets with labels above
group_selector_container = widgets.VBox([group_selector_label, group_selector])
column_selector_container = widgets.VBox([column_selector_label, column_selector])
toggle_buttons_container = widgets.VBox([toggle_buttons_label, toggle_buttons])
lock_button_container = widgets.VBox([lock_button_label, lock_button])

# Display the widgets in a horizontal layout
display(widgets.HBox([group_selector_container, column_selector_container, toggle_buttons_container, lock_button_container]), output)